# IR PROJECT CODE

Members:

*   Omer
*   Lucas
*   Muhamad

# Download and Inspect the Collection

The dataset was created from the Chronicling America collection — over 21 million digitized newspaper pages (1756–1963) curated by the Library of Congress and NEH. They used 39,330 pages (1800–1920), representing 53 US states, to ensure wide geographic and temporal coverage.

Source: https://dl.acm.org/doi/pdf/10.1145/3626772.3657891

GitHub: https://github.com/DataScienceUIBK/ChroniclingAmericaQA?tab=readme-ov-file

In [ ]:
!pip install pyterrier
!pip install pyterrier[java]

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.8/304.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 86.7 MB/s eta 0:00:00
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=d61c135a21a74c5142cef4d79be4f2905418511f0d6394bff8bc84354e816af7
  Stored in directory: /root/.cache/pip/wheels/f6/85/c2/9f0f621def52a1d5db7d29984f81e45f9fb6dfeb1a4eb6e31c
  C

In [ ]:
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/test.json?download=true" -o test.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/train.json?download=true" -o train.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/dev.json?download=true" -o validation.json

import json

files = ["train.json", "validation.json", "test.json"]

for path in files:
    print(f"\n===== {path} =====")
    try:
        with open(path, "r", encoding="utf-8") as f:
            # Read a few hundred characters to see what kind of JSON it is
            head = f.read(500)
            print("Preview of first 500 characters:\n")
            print(head[:500])
        # Try to load only part of the file
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            print(f"\nLoaded {len(data)} items (list).")
            print("Dictionary keys:", list(data[0].keys()))
            print(json.dumps(data[0], indent=2)[:600])
        elif isinstance(data, dict):
            print("\nTop-level is a dictionary. Keys:", list(data.keys()))
            for k, v in data.items():
                if isinstance(v, list):
                    print(f"Key '{k}' contains a list of {len(v)} items.")
                    if v:
                        print("First item keys:", list(v[0].keys()))
                        print(json.dumps(v[0], indent=2)[:600])
                        break
        else:
            print(f"Unexpected top-level type: {type(data)}")
    except Exception as e:
        print(f"Could not parse {path} as JSON: {e}")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1354  100  1354    0     0   3125      0 --:--:-- --:--:-- --:--:--  3127
100 71.5M  100 71.5M    0     0  80.5M      0 --:--:-- --:--:-- --:--:-- 80.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1348  100  1348    0     0   3411      0 --:--:-- --:--:-- --:--:--  3412
100 1315M  100 1315M    0     0  66.9M      0  0:00:19  0:00:19 --:--:-- 39.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1350  100  1350    0     0   6760      0 --:--:-- --:--:-- --:--:--  6783
100 71.8M  100 71.8M    0     0   104M      0 --:--:-- --:--:-- --:--:--  246M

===== train.json =====
Preview of first 500 charact

# Create the Document Collection

To do that, we create a new json file that contains the 'para_id', 'context', 'raw_ocr', 'publication_date' keys, for all para_id in the collection.

para_id: is the id of a paragraph of a news paper page.

In [ ]:
import json
import os

inputs = ["train.json", "validation.json", "test.json"]
output = "document_collection.json"

def load_list_or_empty(path):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        print(f"Skipping {path} because it is missing or empty")
        return []
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            return data
        print(f"Skipping {path} because it is not a list at the top level")
        return []
    except json.JSONDecodeError:
        print(f"Skipping {path} because it is not valid JSON")
        return []

def project(recs):
    out = []
    for r in recs:
        out.append({
            "para_id": r.get("para_id", ""),
            "context": r.get("context", ""),
            "raw_ocr": r.get("raw_ocr", ""),
            "publication_date": r.get("publication_date", "")
        })
    return out

all_recs = []
for p in inputs:
    recs = load_list_or_empty(p)
    print(f"Loaded {len(recs)} records from {p}")
    all_recs.extend(project(recs))

# deduplicate by para_id keeping the first one seen
uniq = {}
for rec in all_recs:
    pid = rec.get("para_id", "")
    if pid and pid not in uniq:
        uniq[pid] = rec

result = list(uniq.values())

with open(output, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"Wrote {len(result)} records to {output}")
print(json.dumps(result[:3], indent=2))

Loaded 439302 records from train.json
Loaded 24111 records from validation.json
Loaded 24084 records from test.json
Wrote 131921 records to document_collection.json
[
  {
    "para_id": "New_Hampshire_18070804_1",
    "context": "Aiscellaneous Repository. From the Albany Register, WAR, OR A PROSPECT OF IT, From recent instances of British Outrage. BY: WILLIAM RAY, Author of the contemplated publication, entitled, \u201cHorrors of Slavery, or the American Turf in Tripoli,\u201d VOTARIES of Freedom, arm! The British Lion roars! Legions of Valor, take th\u2019 alarm\u2014; Rash, rush to guard our shores! Behold the horrid deed\u2014 Your brethren gasping lie! Beneath a tyrant\u2019s hand they bleed\u2014 They groan\u2014they faint\u2014they die. Veterans of seventy-six, Awake the slumbering sword;\u2014 Hearts of your murderous foes transfix\u2014 'Tis vengeance gives the word. Remember Lexington, And Bunker\u2019s tragic hill; \u201cThe same who spilt your blood thereon, Your blood again

## You should check that the collection you have matches that of the paper!

# Create the Test Queries Data Structure

We keep the first 10.000 queries due to memory errors in the free colab version.

To be comparable, please keep the top 10.000 queries for evaluation.

In [ ]:
import json
import re
import unicodedata
import string

input_file = "test.json"
output_file = "test_queries.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

def clean_question(text):
    if not isinstance(text, str):
        return ""
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(rf"[{re.escape(string.punctuation)}]", " ", text)  # remove punctuation
    text = re.sub(r"\s+", " ", text)  # collapse multiple spaces
    return text.strip()

# Extract and clean
queries = [
    {
        "query_id": item.get("query_id", ""),
        "question": clean_question(item.get("question", "")),
    }
    for item in data
]

# Sort by query_id (assuming numeric)
queries = sorted(queries, key=lambda x: int(x["query_id"]) if str(x["query_id"]).isdigit() else x["query_id"])

# Keep only the first 10,000
queries = queries[:10000]

# Save new JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(queries, f, ensure_ascii=False, indent=2)

print(f"Saved {len(queries)} entries to {output_file}")
print(json.dumps(queries[:3], indent=2))

Saved 10000 entries to test_queries.json
[
  {
    "query_id": "test_1",
    "question": "How many lots did Thomas Peirce have"
  },
  {
    "query_id": "test_10",
    "question": "Who gave Hamilton the substance of what he had proposed on the part of General Hamilton"
  },
  {
    "query_id": "test_100",
    "question": "Who informs his FRIENDS and the PUBLIC that he has taken that justly celebrated INN in this city"
  }
]


# Create the Qrels for the test set

In [ ]:
input_file = "test.json"
qrels_file = "test_qrels.json"
answers_file = "test_query_answers.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Build the qrels file: query_id, iteration=0, para_id, relevance=1
qrels = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1
    }
    for item in data
]

# Build the query_answers file: same plus answer and org_answer
query_answers = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1,
        "answer": item.get("answer", ""),
        "org_answer": item.get("org_answer", "")
    }
    for item in data
]

# Save both files
with open(qrels_file, "w", encoding="utf-8") as f:
    json.dump(qrels, f, ensure_ascii=False, indent=2)

with open(answers_file, "w", encoding="utf-8") as f:
    json.dump(query_answers, f, ensure_ascii=False, indent=2)

print(f"Saved {len(qrels)} entries to {qrels_file}")
print(f"Saved {len(query_answers)} entries to {answers_file}")
print("Sample qrels entry:", qrels[0])
print("Sample query_answers entry:", query_answers[0])

Saved 24084 entries to test_qrels.json
Saved 24084 entries to test_query_answers.json
Sample qrels entry: {'query_id': 'test_1', 'iteration': 0, 'para_id': 'New_Hampshire_18030125_16', 'relevance': 1}
Sample query_answers entry: {'query_id': 'test_1', 'iteration': 0, 'para_id': 'New_Hampshire_18030125_16', 'relevance': 1, 'answer': '183', 'org_answer': '183'}


# Retrieval - Good Luck! (YOUR WORK STARTS HERE !)

# Create the Index of the collection



In [ ]:
import os
import pyterrier as pt
import json
import pandas as pd
import shutil

#path index folder
folder_name = "Index"
index_file_name = "terrier_inverted_index"
index_path = os.path.abspath(os.path.join(folder_name, index_file_name))

#init pyTer
if not pt.java.started():
    pt.java.init()

def get_index():
    #check if the index exsists
    properties_file = os.path.join(index_path, "data.properties")
    if os.path.exists(properties_file):
        return pt.IndexFactory.of(index_path)
    print("Index is not found, creating a new Index")
    if not os.path.exists(index_path):
        os.makedirs(index_path, exist_ok=True)
    with open('document_collection.json', 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    #build Data frame and clean the row context field
    df = pd.DataFrame(raw_data)
    df = df.rename(columns={"para_id": "docno", "context": "text"})[["docno", "text", "publication_date"]]

    # Build the index
    # Build the index using the updated IterDictIndexer signature
    # Key parameters now are: meta, text_attrs, meta_reverse, pretokenised, fields, threads
    indexer = pt.IterDictIndexer(
        index_path,
        meta={"docno": 20},            # store docno as metadata (up to 20 characters)
        text_attrs=["text"],           # which field(s) contain the text
        meta_reverse=["docno"],        # enable reverse lookup on docno
        pretokenised=False,
        fields=False,
        threads=1,
    )
    index_ref = indexer.index(df.to_dict(orient="records"))
    return pt.IndexFactory.of(index_ref)
index = get_index()

# Print a simple summary
print("Index location:", index_path)
print("Indexed documents:", index.getCollectionStatistics().getNumberOfDocuments())



terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-assemblies/5.11/terrier-assemblies-5.11-jar-with-dependenci…

Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-python-helper/0.0.8/terrier-python-helper-0.0.8.jar:   0%| …

Done


Java started and loaded: pyterrier.java.colab, pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


Index is not found, creating a new Index
13:54:22.695 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (Michigan_18370308_24) - further warnings are suppressed
13:54:37.871 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents
Index location: /content/Index/terrier_inverted_index
Indexed documents: 131921


In [ ]:
# Retrieve collection statistics
stats = index.getCollectionStatistics()

print("Terrier Collection Statistics")
print("--------------------------------")
print(f"Indexed documents:        {stats.getNumberOfDocuments()}")
print(f"Unique terms (vocabulary): {stats.getNumberOfUniqueTerms()}")
print(f"Total tokens:             {stats.getNumberOfTokens()}")
print(f"Average document length:  {stats.getAverageDocumentLength():.2f}")

Terrier Collection Statistics
--------------------------------
Indexed documents:        131921
Unique terms (vocabulary): 236646
Total tokens:             15575099
Average document length:  118.06


# 1st stage retrieval (skeleton code)

In [ ]:
# First stage retriver with BM25
bm25 = pt.terrier.Retriever(index, wmodel="BM25")
tf_idf = pt.terrier.Retriever(index, wmodel="TF_IDF")

In [ ]:
from pyterrier.measures import P, MAP, R, nDCG
import pyterrier as pt # Ensure pyterrier is imported as pt

# First stage retriver with BM25
bm25 = pt.terrier.Retriever(index, wmodel="BM25")
tf_idf = pt.terrier.Retriever(index, wmodel="TF_IDF")

# Define pipelines with truncation for faster experimentation using the topK operator
bm25_pipeline = bm25 % 100
tf_idf_pipeline = tf_idf % 100

#testing the engine
query = "Europe"
bm25_results = bm25_pipeline.search(query)
tf_idf_results = tf_idf_pipeline.search(query)

#show first 10 results
print('-------------------- bm25 results (top 100) --------------------\n',bm25_results.head(10))
print('-------------------- tf_idf results (top 100) --------------------\n',tf_idf_results.head(10))

# Convert queries and qrels to pandas DataFrames
queries_df = pd.DataFrame(queries)
qrels_df = pd.DataFrame(qrels)

# Rename columns to match PyTerrier's expected names for evaluation
queries_df = queries_df.rename(columns={"query_id": "qid", "question": "query"})
qrels_df = qrels_df.rename(columns={"query_id": "qid", "para_id": "docno"})

# WARNING taking the whole queries will take a long time
# --- Use a subset of queries for faster computation,  ---
num_queries_subset = 200 # You can adjust this number

queries_subset_df = queries_df.head(num_queries_subset)
qrels_subset_df = qrels_df[qrels_df['qid'].isin(queries_subset_df['qid'])].copy()

experiment_results = pt.Experiment(
    [bm25_pipeline, tf_idf_pipeline],
    queries_subset_df, # Use the subset of queries
    qrels_subset_df,   # Use the filtered qrels for the subset
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["BM25_top100","TF-IDF_top100"],
)

display(experiment_results)

-------------------- bm25 results (top 100) --------------------
   qid   docid                     docno  rank      score   query
0   1  122024      New_York_19190610_27     0  10.233243  Europe
1   1   61883         Texas_18771215_13     1   9.759719  Europe
2   1    9637      Maryland_18401110_20     2   9.730148  Europe
3   1  110750      Nebraska_19020225_26     3   9.715430  Europe
4   1   36539  Rhode_Island_18511030_25     4   9.642500  Europe
5   1   47808      Illinois_18680907_40     5   9.578485  Europe
6   1  100207         Kansas_18970902_5     6   9.570657  Europe
7   1   65964     Tennessee_18740915_14     7   9.556417  Europe
8   1   29371        Hawaii_18520522_33     8   9.509314  Europe
9   1   38017     Tennessee_18590726_13     9   9.499877  Europe
-------------------- tf_idf results (top 100) --------------------
   qid   docid                     docno  rank     score   query
0   1  122024      New_York_19190610_27     0  5.641964  Europe
1   1   61883         T

,name,AP,P@1,P@5,P@10,R@5,R@10,nDCG@5,nDCG@10
0,BM25_top100,0.681958,0.605,0.157,0.0840,0.785,0.840,0.697817,0.716238
1,TF-IDF_top100,0.688504,0.610,0.156,0.0835,0.780,0.835,0.701298,0.719701


MESSAGE: THE CODE BELOW IS SUPPOSED TO RETRIEVE THE WHOLE QUERIES BUT BECAUSE OF COMPUTATION SPEED, IT DIDN'T GIVE A RESULT IN A REASONABLE TIME

In [ ]:
"""
experiment_results = pt.Experiment(
    [bm25,tf_idf],
    queries_df, # Use all the queries
    qrels_df,   # Use all the qrels
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["BM25","TF-IDF"],
)"""

'\nexperiment_results = pt.Experiment(\n    [bm25,tf_idf],\n    queries_df, # Use all the queries\n    qrels_df,   # Use all the qrels\n    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],\n    names=["BM25","TF-IDF"],\n)'

In [ ]:
res_qtitles_tfidf = tfidf.transform(queries_titles[['qid','query']])# ASPIRE takes res
res_qtitles_bm25 = bm25.transform(queries_titles[['qid','query']])# ASPIRE takes res

display(res_qtitles_bm25)

# 2nd stage retrieval (skeleton code)

In [ ]:
from pyterrier.measures import P, MAP, R, nDCG
import pyterrier as pt # Ensure pyterrier is imported as pt

# First stage retriver with BM25
bm25 = pt.terrier.Retriever(index, wmodel="BM25")
tf_idf = pt.terrier.Retriever(index, wmodel="TF_IDF")

# Define pipelines with truncation for faster experimentation using the topK operator
bm25_pipeline = bm25 % 100
tf_idf_pipeline = tf_idf % 100

# Define RM3 re-ranker for query expansion
# It uses the top documents from initial retrieval to expand the query
rm3 = pt.rewrite.RM3(index, fb_docs=10, fb_terms=10)

# Create pipelines with RM3 re-ranker
# The output of rm3 (rewritten query) needs to be passed back to a retriever
bm25_rm3_pipeline = (bm25 % 100) >> rm3 >> (bm25 % 100)
tf_idf_rm3_pipeline = (tf_idf % 100) >> rm3 >> (tf_idf % 100)

#testing the engine
query = "Europe"
bm25_rm3_results = bm25_rm3_pipeline.search(query)
tf_idf_rm3_results = tf_idf_rm3_pipeline.search(query)

#show first 10 results
print('-------------------- bm25 + RM3 results (top 100) --------------------\n',bm25_rm3_results.head(10))
print('-------------------- tf_idf + RM3 results (top 100) --------------------\n',tf_idf_rm3_results.head(10))

# Convert queries and qrels to pandas DataFrames
queries_df = pd.DataFrame(queries)
qrels_df = pd.DataFrame(qrels)

# Rename columns to match PyTerrier's expected names for evaluation
queries_df = queries_df.rename(columns={"query_id": "qid", "question": "query"})
qrels_df = qrels_df.rename(columns={"query_id": "qid", "para_id": "docno"})

# --- Use a subset of queries for faster computation,  ---
num_queries_subset = 200 # You can adjust this number

queries_subset_df = queries_df.head(num_queries_subset)
qrels_subset_df = qrels_df[qrels_df['qid'].isin(queries_subset_df['qid'])].copy()

experiment_results = pt.Experiment(
    [bm25_rm3_pipeline, tf_idf_rm3_pipeline],
    queries_subset_df, # Use the subset of queries
    qrels_subset_df,   # Use the filtered qrels for the subset
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["BM25_top100", "TF-IDF_top100", "BM25+RM3_top100", "TF-IDF+RM3_top100"],
)

display(experiment_results)

-------------------- bm25 results (top 100) --------------------
   qid   docid                     docno  rank      score   query
0   1  122024      New_York_19190610_27     0  10.233243  Europe
1   1   61883         Texas_18771215_13     1   9.759719  Europe
2   1    9637      Maryland_18401110_20     2   9.730148  Europe
3   1  110750      Nebraska_19020225_26     3   9.715430  Europe
4   1   36539  Rhode_Island_18511030_25     4   9.642500  Europe
5   1   47808      Illinois_18680907_40     5   9.578485  Europe
6   1  100207         Kansas_18970902_5     6   9.570657  Europe
7   1   65964     Tennessee_18740915_14     7   9.556417  Europe
8   1   29371        Hawaii_18520522_33     8   9.509314  Europe
9   1   38017     Tennessee_18590726_13     9   9.499877  Europe
-------------------- tf_idf results (top 100) --------------------
   qid   docid                     docno  rank     score   query
0   1  122024      New_York_19190610_27     0  5.641964  Europe
1   1   61883         T

,name,AP,P@1,P@5,P@10,R@5,R@10,nDCG@5,nDCG@10
0,BM25_top100,0.681958,0.605,0.157,0.0840,0.785,0.840,0.697817,0.716238
1,TF-IDF_top100,0.688504,0.610,0.156,0.0835,0.780,0.835,0.701298,0.719701
2,BM25+RM3_top100,0.656784,0.595,0.145,0.0780,0.725,0.780,0.663732,0.681379
3,TF-IDF+RM3_top100,0.652954,0.585,0.145,0.0780,0.725,0.780,0.660735,0.678705


### Using ASPIRE (TEMPORARY)


PyTerrier : https://pyterrier.readthedocs.io/en/latest/experiments.html

ASPIRE https://github.com/GiorgosPeikos/ASPIRE

ONLINE: https://aspire-ir-eval.streamlit.app/



Remember the res files are actually the rankings created by our retrieval model!

We downloading it to use another tool for data analysis. The tool is called ASPIRE.

ASPIRE requires a different input format!

In [ ]:
import csv
import pandas as pd # Ensure pandas is imported
import xml.etree.ElementTree as ET
from xml.dom import minidom

# ----- RESULTS (TF-IDF, BM25, etc.) -----
def save_res(df, run_name, out_path, sep="\t"):
    res = df.copy()  # keep original untouched
    if "docid" in res.columns and "docno" not in res.columns:
        res = res.rename(columns={"docid": "docno"})  # normalize doc id
    # The 'query' column is often added by pyterrier for inspection but not needed in output
    res = res.drop(columns=["query", "query_0"], errors="ignore") # remove query text, keep qid
    res["iteration"] = "Q0"  # fixed iteration value
    res["experiment_id"] = run_name  # label for the run
    res = res.rename(columns={"qid": "query_id", "docno": "doc_id"})  # final names
    # Ensure score is numeric for sorting, if needed, though pyterrier usually keeps it
    res["score"] = pd.to_numeric(res["score"])
    # Sort by score for ranking within each query_id
    res = res.sort_values(by=["query_id", "score"], ascending=[True, False])
    res["rank"] = res.groupby("query_id")["score"].rank(ascending=False, method="first").astype(int)
    res = res[["query_id", "iteration", "doc_id", "rank", "score", "experiment_id"]]  # correct column order
    res.to_csv(out_path, sep=sep, index=False, header=False)  # write file


# Save the results to CSV files for ASPIRE
save_res(full_bm25_results, "BM25_top100", "results_bm25_top100.csv")
save_res(full_tf_idf_results, "TF-IDF_top100", "results_tf_idf_top100.csv")


print("Generated results CSVs for BM25 and TF-IDF.")

# ----- QUERIES -----
# root with task attribute
# Renaming 'qid' to 'query_id' for consistency if needed, but XML expects 'number' attribute
root = ET.Element("topics", attrib={"task": "Chronicling America QA"})  # create root element

# add one <topic number=\"...\"> per row
for _, row in queries_df.iterrows():  # iterate over dataframe rows (using queries_df)
    topic = ET.SubElement(root, "topic", attrib={"number": str(row["qid"])})  # topic node
    topic.text = str(row["query"])  # topic content

# pretty print
rough_xml = ET.tostring(root, encoding="utf-8")  # raw bytes
pretty_xml = minidom.parseString(rough_xml).toprettyxml(indent="  ", encoding="utf-8")  # formatted bytes

# write to file
with open("queries.xml", "wb") as f:  # save xml with declaration
    f.write(pretty_xml)

print("Generated queries.xml file.")

# ----- QRELS -----
qrels_out = qrels_df.copy() # Use qrels_df which is available in kernel state
qrels_out = qrels_out.drop(columns=["iteration"], errors="ignore") # iteration from qrels_df is int, ASPIRE wants string 'Q0'
if "docid" in qrels_out.columns and "docno" not in qrels_out.columns:
    qrels_out = qrels_out.rename(columns={"docid": "docno"})
qrels_out = qrels_out.rename(columns={"qid": "query_id", "docno": "doc_id"})
qrels_out["iteration"] = "0" # Ensure '0' string for ASPIRE format
qrels_out = qrels_out[["query_id", "iteration", "doc_id", "relevance"]]
qrels_out.to_csv("qrels.csv", sep=" ", index=False, header=False)

print("Generated qrels.csv file.")

print("All files generated for ASPIRE compatibility.")

Generated results CSVs for BM25 and TF-IDF.
Generated queries.xml file.
Generated qrels.csv file.
All files generated for ASPIRE compatibility.


In [ ]:
# To use ASPIRE, we need to download the res files, the qrels and queries in the
# required dataformat and use the interface or use it locally.

# Res file Expected format: query_id, iteration (i.e. Q0), doc_id, rank, score, experiment_id
# Queries Expected format: query_id, query
# Qrels Expected format: query_id, iteration (i.e. Q0), doc_id, relevance (without header row)

# DEMO TIME!